In [1]:
import sys
sys.path.insert(0,'..')

from src.models import *
from src.loss_functions import *
from src.noise import *
from src.metrics import *
from src.plotting import *
from src.generate_data import *
from src.real_data import *

import sklearn
import pandas as pd

from scipy.stats import bernoulli

from operator import xor
from sklearn.preprocessing import StandardScaler

In [11]:
from folktables import ACSDataSource, ACSEmployment


data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["AL"], download=True)



size = 2500
acs_data_sample = acs_data.sample(n=size, random_state=2024)

features, label, group = ACSEmployment.df_to_numpy(acs_data_sample)

In [12]:
# Create a StandardScaler object
scaler = StandardScaler()

# Fit and transform the data
features = scaler.fit_transform(features)

In [13]:
features.shape

(2500, 16)

In [14]:
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    features, label, group, test_size=0.2, random_state=2024)

In [15]:
X_train.shape

(2000, 16)

In [16]:
noise_levels = np.linspace(0.0, 0, num=5)
noise_type = "class_independent"
fixed_class = 1
fixed_noise = 0.0

model_type = "LR"
uncertainty_type = "forward"

max_iter = 100
m = 100
d= 16



X_train = X_train
y_train = y_train.astype(int)

y_test = y_test.astype(int)

np.unique(y_train, return_counts=True)

p1 = np.sum(y_train)/len(y_train)
p_y_x = np.array([1-p1,p1])
p_y_x

array([0.586, 0.414])

In [ ]:
metrics = simulate_noise_and_train_model(noise_levels, m, max_iter, d,  X_train, y_train, X_test, y_test, p_y_x = p_y_x, noise_type = noise_type, model_type = model_type, uncertainty_type=uncertainty_type,  fixed_class=fixed_class, fixed_noise=fixed_noise)
    

  0%|          | 0/5 [00:00<?, ?it/s]

[[1 1 0 ... 1 0 1]
 [1 1 0 ... 1 0 1]
 [0 1 0 ... 1 0 1]
 ...
 [1 1 0 ... 1 0 1]
 [1 1 0 ... 1 0 1]
 [1 1 0 ... 1 0 1]]
Typical Rate:  1.0
[[1 1 0 ... 1 0 1]
 [0 1 0 ... 1 0 1]
 [1 1 0 ... 1 0 1]
 ...
 [1 1 0 ... 1 0 1]
 [1 1 0 ... 1 0 1]
 [1 1 0 ... 1 0 1]]
Typical Rate:  1.0
[[0 1 0 ... 1 0 1]
 [1 1 0 ... 1 0 1]
 [1 1 0 ... 1 0 1]
 ...
 [1 1 0 ... 1 0 1]
 [1 1 0 ... 1 0 1]
 [0 1 0 ... 1 0 1]]
Typical Rate:  1.0


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def plot_metric(metrics, metric, ax=None):
    # Prepare the data
    loss_functions = list(metrics.data.keys())
    noise_levels = list(set(noise for lf in loss_functions for noise in metrics.data[lf].keys()))
    rows = []

    for loss_function in loss_functions:
        for noise in noise_levels:
            if noise in metrics.data[loss_function]:
                for value in metrics.data[loss_function][noise][metric]:
                    rows.append({
                        'Noise Rate': noise,
                        'Metric': value,
                        'Loss Function': loss_function
                    })
    
    # Create a DataFrame suitable for Seaborn
    df = pd.DataFrame(rows)
    
    # Create the plot
    ax = ax or plt.gca()
    sns.boxplot(x='Noise Rate', y='Metric', hue='Loss Function', data=df, ax=ax, palette='Set2', showfliers=False)
    sns.stripplot(x='Noise Rate', y='Metric', hue='Loss Function', data=df, ax=ax, dodge=True, jitter=True, color='black', size=5, alpha=0.8)
    
    # Add legend and adjust plot details
    handles, labels = ax.get_legend_handles_labels()
    l = plt.legend(handles[:len(loss_functions)], labels[:len(loss_functions)], title='Loss Function')
    ax.add_artist(l)
    
    # Set plot title and labels if ax is None
    if ax is None:
        ax.set_title(f'{metric} by Noise Rate Across Loss Functions')
        ax.set_xlabel('Noise Rate')
        ax.set_ylabel(metric.capitalize())
        plt.show()


# Usage:
plot_metric(metrics,  'disagreement_train')
# or to plot on an existing axes:
# fig, ax = plt.subplots()
# plot_metric(metrics, 'BCE', 'clean_test_acc', ax=ax)
